In [ ]:
import math
import numpy as np
# import tensorflow as tf
import cv2
import scipy.sparse as ssp
from scipy import ndimage
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
def detect_points_with_masks(frame):
    """
    フレームから白い点を検出し、それぞれの点に対応するマスク（輝度分布）を取得する。
    
    Parameters:
        frame (ndarray): 入力フレーム（2D配列）。
    
    Returns:
        centers (ndarray): 各点の重心座標の配列（形状: (N, 2)）。
        masks (list of ndarray): 各点のマスク（輝度分布を含む、形状: (H, W)）。
    """
    binary = (frame >= 0).astype(np.uint8)
    
    # 点のラベリング
    labeled, num_features = ndimage.label(binary)
    
    centers = []
    masks = []
    
    for i in range(1, num_features + 1):
        mask = (labeled == i).astype(np.float32)
        min_pixels = 2
        if np.sum(mask) < min_pixels:
            continue  # 小さすぎる点は無視
        
        # 元のフレームの輝度をマスク
        mask_with_brightness = mask * frame
        
        # 重心の計算
        center = ndimage.center_of_mass(mask)
        centers.append(center)
        masks.append(mask_with_brightness)
    
    centers = np.array(centers)  # 形状: (N, 2)
    return centers, masks

def compute_motion_vectors(prev_centers, next_centers, max_distance=10):
    """
    各点の動きベクトルを計算する。
    最も近い点を対応付けとして動きベクトルを計算する。
    
    Parameters:
        prev_centers (ndarray): 前のフレームの点の重心座標（形状: (N, 2)）。
        next_centers (ndarray): 次のフレームの点の重心座標（形状: (M, 2)）。
        max_distance (float): 対応付けを許容する最大距離。
    
    Returns:
        motion_vectors (ndarray): 各前フレームの点に対応する動きベクトル（形状: (N, 2)）。
        matched_next_indices (list): 各前フレームの点に対応する次フレームの点のインデックス。対応がない場合はNone。
    """
    motion_vectors = []
    matched_next_indices = []
    used_next = set()
    
    # next_centersが空の場合、すべての動きベクトルをゼロに
    if next_centers.size == 0:
        for _ in prev_centers:
            motion_vectors.append([0.0, 0.0])
            matched_next_indices.append(None)
        return np.array(motion_vectors), matched_next_indices
    
    for p in prev_centers:
        distances = np.linalg.norm(next_centers - p, axis=1)
        min_idx = np.argmin(distances)
        min_dist = distances[min_idx]
        
        if min_dist <= max_distance and min_idx not in used_next:
            used_next.add(min_idx)
            vector = next_centers[min_idx] - p  # (dy, dx)
            matched_next_indices.append(min_idx)
        else:
            vector = np.array([0.0, 0.0])  # 対応が見つからない場合は動かない
            matched_next_indices.append(None)
        
        motion_vectors.append(vector)
    
    return np.array(motion_vectors), matched_next_indices  # 形状: (N, 2), list of matched indices or None

def interpolate_masks(current_masks, next_masks, motion_vectors, matched_next_indices, alpha, frame_shape):
    """
    各点のマスクを補間位置に移動させて新しいマスクを生成する。
    各点について、前フレームのマスクを1/2動きベクトル分移動させ、
    次フレームのマスクを-1/2動きベクトル分移動させたものをブレンドする。
    
    Parameters:
        current_masks (list of ndarray): 前のフレームのマスク（輝度分布）。
        next_masks (list of ndarray): 次のフレームのマスク（輝度分布）。
        motion_vectors (ndarray): 各点の動きベクトル（形状: (N, 2)）。
        matched_next_indices (list): 各前フレームの点に対応する次フレームの点のインデックス。対応がない場合はNone。
        alpha (float): 補間係数（通常は0.5）。
        frame_shape (tuple): 補間フレームの形状（高さ, 幅）。
    
    Returns:
        interp_frame (ndarray): 補間フレーム（2D配列）。
    """
    interp_frame = np.zeros(frame_shape, dtype=np.float32)
    num_points = len(current_masks)
    
    for i in range(num_points):
        vector = motion_vectors[i]
        half_vector = vector * 0.5
        
        # 前フレームのマスクを1/2動きベクトル分移動
        M_current = np.float32([[1, 0, half_vector[1]], [0, 1, half_vector[0]]])  # [ [1, 0, dx], [0, 1, dy] ]
        shifted_current = cv2.warpAffine(current_masks[i], M_current, (frame_shape[1], frame_shape[0]),
                                            flags=cv2.INTER_LINEAR, borderValue=0)
        
        # 次フレームのマスクを-1/2動きベクトル分移動
        if matched_next_indices[i] is not None and matched_next_indices[i] < len(next_masks):
            next_mask = next_masks[matched_next_indices[i]]
            M_next = np.float32([[1, 0, -half_vector[1]], [0, 1, -half_vector[0]]])
            shifted_next = cv2.warpAffine(next_mask, M_next, (frame_shape[1], frame_shape[0]),
                                            flags=cv2.INTER_LINEAR, borderValue=0)
        else:
            # 対応する次フレームのマスクがない場合はゼロマスクを使用
            shifted_next = np.zeros(frame_shape, dtype=np.float32)
        
        # マスクのブレンド（平均）
        blended_mask = (shifted_current + shifted_next) / 2.0
        
        # 補間フレームに加算
        interp_frame += blended_mask
    
    return interp_frame

def interpolate_frames(frames, num_interpolations=1):
    """
    フレーム補間を行う。
    各フレーム間に指定された数の補間フレームを挿入する。
    
    Parameters:
        frames (ndarray): 元のフレーム配列（形状: (num_frames, H, W)）。
        num_interpolations (int): 各フレーム間に挿入する補間フレームの数。
    
    Returns:
        interpolated (ndarray): 補間後のフレーム配列（形状: new_num_frames, H, W）。
    """
    interpolated = []
    num_frames = frames.shape[0]
    height, width = frames[0].shape
    
    for t in range(num_frames - 1):
        current_frame = frames[t]
        next_frame = frames[t + 1]
        
        # 点の検出とマスクの取得
        current_centers, current_masks = detect_points_with_masks(current_frame)
        next_centers, next_masks = detect_points_with_masks(next_frame)
        
        # 動きベクトルの計算と対応付けの取得
        motion_vectors, matched_next_indices = compute_motion_vectors(current_centers, next_centers)
        
        # 現在のフレームを追加
        # interpolated.append(current_frame)
        
        # 点が存在しない場合の処理
        if len(current_centers) == 0 or len(next_centers) == 0:
            # 補間フレームとして黒いフレームを追加
            for k in range(1, num_interpolations + 1):
                interp_frame = np.zeros((height, width), dtype=np.float32)
                interpolated.append(interp_frame)
            continue
        
        # 各補間フレームの生成
        for k in range(1, num_interpolations + 1):
            # 補間係数（通常は0.5）
            alpha = k / (num_interpolations + 1)
            
            # 補間フレームの生成
            interp_frame = interpolate_masks(current_masks, next_masks, motion_vectors, matched_next_indices, alpha, (height, width))
            
            interpolated.append(interp_frame)
    
    # 最後のフレームを追加
    # interpolated.append(frames[-1])
    return np.array(interpolated)

In [ ]:
def matrix_to_tensor(H, m, n):
    return H.reshape(m, m, n, n)


def tensor_to_matrix(tensor, m, n):
    return tensor.reshape(m * m, n * n)


n = 128
_m = 128
# DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
DATA_PATH = "../data"
EXP_DATE = "241105"
H_SETTING = "p-5_lmd-100_m-128"
H_mat = np.load(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_{H_SETTING}.npy")
print("H shape:", H_mat.shape, "type(H):", type(H_mat), "H.dtype:", H_mat.dtype)
# しきい値処理
H_mat[np.abs(H_mat) < 0.01] = 0
H_ten = matrix_to_tensor(H_mat, _m, n)

In [ ]:
# フレーム補間
m = 2 * _m - 1  # 255
interpolated_tensor = np.zeros((m, m, n, n), dtype=np.float32)
for i in range(0, m, 2):
    frames = H_ten[i // 2, :, :, :]
    interpolated_frames = interpolate_frames(frames)
    for j in range(m):
        if j % 2 == 0:
            interpolated_tensor[i, j, :, :] = H_ten[i // 2, j // 2, :, :]
        else:
            interpolated_tensor[i, j, :, :] = interpolated_frames[j // 2, :, :]
for j in range(0, m):
    frames = interpolated_tensor[::2, j, :, :]
    interpolated_frames = interpolate_frames(frames)
    for i in range(1, m, 2):
        interpolated_tensor[i, j, :, :] = interpolated_frames[i // 2, :, :]

print("interpolated_tensor.shape:", interpolated_tensor.shape, "dtype:", interpolated_tensor.dtype)
interpolated_matrix = tensor_to_matrix(interpolated_tensor, m, n)

In [ ]:
np.save(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_int_{H_SETTING}.npy", interpolated_matrix)
print(f"Saved {DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_int_{H_SETTING}.npy")

In [ ]:
SAMPLE_NAME = "Cameraman"
sample_image = Image.open(f"{DATA_PATH}/sample_image{n}/{SAMPLE_NAME}.png").convert("L")
sample_image = np.asarray(sample_image).flatten() / 255

Hf = interpolated_matrix @ sample_image
# Hf_img = Hf.reshape(m, m)
Hf_img = np.asnumpy(Hf.reshape(m, m), Image.resampling.BICUBIC)
Hf_img = np.clip(Hf_img, 0, 1)
print("Hf shape:", Hf_img.shape)
Hf_pil = Image.fromarray((Hf_img * 255).astype(np.uint8), mode="L")

fig, ax = plt.subplots(figsize=Hf_img.shape[::-1], dpi=1, tight_layout=True)
ax.imshow(Hf_pil, cmap="gray")
ax.axis("off")
plt.show()
FILENAME = f"{SAMPLE_NAME}_{H_SETTING}.png"
# fig.savefig(f"{DIRECTORY}/{FILENAME}", dpi=1)
# print(f"Saved {DIRECTORY}/{FILENAME}")

In [ ]:
J = 128
interpolated_frames = interpolated_tensor[:, J, :, :]
print("interpolated_frames.shape:", interpolated_frames.shape)
# 表示したいフレームの範囲を設定
start_index = 60
end_index = 84
frames_to_display = interpolated_frames[start_index:end_index + 1]

num_frames = len(frames_to_display)
cols = 5  # 列数を設定（必要に応じて変更可能）
rows = math.ceil(num_frames / cols)  # 行数を自動計算

# サブプロットを作成
fig, axs = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))

# サブプロットの軸を1次元にフラット化してループ処理
for idx, ax in enumerate(axs.flat):
    if idx < num_frames:
        frame = frames_to_display[idx]
        img = ax.imshow(frame, cmap='gray')
        plt.colorbar(img, ax=ax)
        ax.set_title(f'Frame {start_index + idx}')
    else:
        ax.axis('off')  # 使用しないサブプロットを非表示に

In [ ]:
import imageio
import io

def create_heatmap_gif(data, output_path, duration=0.5):
    """
    m×n×nのnumpy配列をヒートマップのGIFに変換する。

    Parameters:
    - data: ndarray, shape (m, n, n)
    - output_path: str, 保存するGIFのパス
    - duration: float, 各フレームの表示時間（秒）
    """
    # データが3次元か確認
    if data.ndim != 3:
        raise ValueError("入力データは3次元のnumpy配列である必要があります。")

    m, n1, n2 = data.shape
    if n1 != n2:
        raise ValueError("各スライスはn×nの形状である必要があります。")

    # テンソルの最大値と最小値を計算
    vmin = np.min(data)
    vmax = np.max(data)

    images = []  # GIFに追加する画像のリスト

    # 図と軸を設定
    fig, ax = plt.subplots(figsize=(6, 6))
    # plt.axis('off')  # 軸を非表示

    # ヒートマップを初期化
    cax = ax.imshow(data[0], cmap='gray', vmin=vmin, vmax=vmax)
    # カラーバーを一度だけ追加
    cbar = fig.colorbar(cax, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=8)
    title = ax.set_title(f'Frame 1/{m}', fontsize=10)

    for i in range(m):
        # ヒートマップのデータを更新
        cax.set_data(data[i])
        # タイトルを更新
        title.set_text(f'Frame {i+1}/{m}')

        # レイアウトを固定
        fig.tight_layout()

        # 画像をバッファに保存
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = imageio.v2.imread(buf)
        images.append(image)
        buf.close()

    plt.close(fig)  # 図を閉じる

    # GIFとして保存
    imageio.mimsave(output_path, images, duration=duration)
    print(f"GIFが保存されました: {output_path}")

create_heatmap_gif(interpolated_frames, f"{DATA_PATH}/{EXP_DATE}/interpolated_j{J}_video.gif", duration=0.2)